In [ ]:
!pip install langchain==0.0.208 deeplake openai tiktoken

# The LLMs

In [1]:
import os
from langchain.llms import OpenAI

In [3]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.

llm = OpenAI(model="text-davinci-003", temperature=0.9)


In [4]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Begin with a 10 minutes light jog or brisk walk.
2. Follow it up with 30 minutes of interval training. Start with 30 seconds of moderate to high-intensity running, followed by 30 seconds rest. Do this at least 8 times.
3. After the interval training, jog for 15 minutes.
4. Cool down with a 5-minute walk.
5. Repeat this routine at least 3-4 times per week.

Additionally, you can add some outdoor activities, such as cycling, swimming, or hiking. You could also add functional exercises such as walking up stairs or hills, using additional bodyweight exercises such as mountain climbers and jumping jacks.


# The Chains

In [5]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)


In [6]:
# Run the chain only specifying the input variable.
print(chain.run("uniquie data science company name that doesn't have data included int"))



IdeaFlux Data Solutions


# The Memory

In [7]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:



> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!
Human: What can you do?
AI:



> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!
Human: What can you do?
AI:  I can help you with a variety of tasks, such as scheduling appointments, setting reminders, and providing information. I'm also able to answer questions about topics like current events, sports, and entertainment. I'm always learning new things, so I'm sure I can help you with whatever you need.
Human: How can you help me with da


> Finished chain.
memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Tell me about yourself.', additional_kwargs={}, example=False), AIMessage(content=" Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!", additional_kwargs={}, example=False), HumanMessage(content='What can you do?', additional_kwargs={}, example=False), AIMessage(content=" I can help you with a variety of tasks, such as scheduling appointments, setting reminders, and providing information. I'm also able to answer questions about topics like current events, sports, and entertainment. I'm always learning new things, so I'm sure I can help you with whatever you need.", additional_kwargs={}, example=False), HumanMessage(content='How can you help me with 

# Deep Lake VectorStore

In [ ]:
!pip install deeplake

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "harbidel"
my_activeloop_dataset_name = "LangChain_Practice"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://harbidel/LangChain_Practice already exists, loading from the storage


100%|██████████| 2/2 [00:04<00:00,  2.05s/it]
\

Dataset(path='hub://harbidel/LangChain_Practice', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
   text       text      (6, 1)      str     None   


['cc3a77da-706e-11ee-bd30-0242ac1c000c',
 'cc3a7942-706e-11ee-bd30-0242ac1c000c']

In [9]:
# creating a retrievalQA

retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [10]:
# reate an agent that uses the RetrievalQA chain as a tool

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [11]:
# Use the agent to ask question

response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?


Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought:

 I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [12]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://harbidel/LangChain_Practice already exists, loading from the storage


100%|██████████| 2/2 [00:00<00:00,  2.49it/s]
/

Dataset(path='hub://harbidel/LangChain_Practice', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (8, 1536)  float32   None   
    id        text      (8, 1)      str     None   
 metadata     json      (8, 1)      str     None   
   text       text      (8, 1)      str     None   


['ecab3ed2-706e-11ee-bd30-0242ac1c000c',
 'ecab4026-706e-11ee-bd30-0242ac1c000c']

In [13]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [14]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...


 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?


Observation:  Michael Jordan was born on 17 February 1963.
Thought:

 I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


In [15]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [16]:
# Initialize LLM and set temp to zero

llm = OpenAI(model="text-davinci-003", temperature=0)

In [17]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

In [18]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [19]:
# creat an agent that uses our Google search tool

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=6)

In [23]:
# Check the response

response = agent("What's the latest news about LLMs?")
print(response['output'])



> Entering new  chain...
 I should search for recent news articles about LLMs.
Action: google-search
Action Input: "LLM news"
Observation: Sep 13, 2023 ... Hugging Face announced Falcon 180B, the most powerful open source LLM ever. It rivals Google's Palm 2... And it has no guardrails. Jul 24, 2023 ... Generative AI News – Llama 2 Shakes up LLM Market, GPT-4 May Not Be Degrading, Apple GPT, Wix, AP, AI21, SAP, and More – Voicebot Podcast 340. Sign up to get the latest news about large language models (LLM) technologies from NVIDIA. Jul 25, 2023 ... TechScape: Will Meta's open-source LLM make AI safer – or put it into the wrong hands? ... news in the field has gone from weekly to almost daily. Jul 11, 2023 ... ITPro Today: IT News, How-Tos, Trends, Case Studies, Career · Search · Log In · Register · NEWSLETTER SIGN-UP. Recent. Recent. Accidental ... Sep 11, 2023 ... AskIt offers type-guided output control, template-based function definitions, and a unified interface that bridges the g

# Tools in LangChain

In [24]:
# import libraries

from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [25]:
# We then instantiate a  LLMChain specifically for text summarization.

llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [26]:
# we create the tools that our agent will use.

# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [27]:
# create our agent that leverages two tools.

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [28]:
# Let’s run the agent with a question about summarizing the latest news about the Mars rover.

response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I need to find the latest news about the Mars rover and then summarize it.
Action: Search
Action Input: Latest news about the Mars rover
Observation: Data sent by the spacecraft before it retired last December has provided new details about how fast the planet fast rotates and how much it wobbles. Dec 15, 2021 ... Mars Sample Return is going to have great stuff to choose from!” Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. The multi-mission Mars ... NASA's Curiosity Mars rover captured these clouds just after sunset on March 19, 2021, ... Curiosity rover finds water-carved 'book' rock on Mars (photo) ... A book-shaped rock on Mars was likely carved by ancient water running through the area, ... NASA's Mars 2020 Perseverance rover will look for signs of past microbial life, cache rock and soil samples, and prepare for future human exploration. Curiosity Rover · Nasa's Curiosity Mars rover can provide a lot of information about the Red Planet, but · An


Observation: cht as the "Mars wobble," the data sent by the spacecraft before it retired last December has provided new details about how fast the planet rotates and how much it wobbles.

NASA's Curiosity Mars rover has been providing a wealth of information about the Red Planet since its launch in 2021. Recently, data sent by the spacecraft before it retired last December has revealed new details about the planet's rotation and wobble. This "Mars wobble" was informally dubbed by Hecht and provides insight into the planet's speed of rotation and the amount of wobble it experiences. In addition, the Mars Sample Return mission is set to launch in 2021, and the Mars 2020 Perseverance rover will look for signs of past microbial life and cache rock and soil samples. The Curiosity rover has also discovered evidence of ancient lakes on the Red Planet.
Thought:

 I now know the final answer
Final Answer: The Curiosity rover has been providing a wealth of information about the Red Planet since its launch in 2021, including new details about the planet's rotation and wobble, as well as evidence of ancient lakes. The Mars Sample Return mission is set to launch in 2021, and the Mars 2020 Perseverance rover will look for signs of past microbial life and cache rock and soil samples.

> Finished chain.
The Curiosity rover has been providing a wealth of information about the Red Planet since its launch in 2021, including new details about the planet's rotation and wobble, as well as evidence of ancient lakes. The Mars Sample Return mission is set to launch in 2021, and the Mars 2020 Perseverance rover will look for signs of past microbial life and cache rock and soil samples.
